<a href="https://colab.research.google.com/github/luizcalabria/redes_neurais_pos/blob/main/Otimiza%C3%A7%C3%A3o_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [ ]:
import random
import numpy as np

np.random.seed(10)
random.seed(10)

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [ ]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = sigmoid(np.dot(w, x)+b) #net = (∑wx+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None, lmbd=0, otimizacao=None, momentum=0.8):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂, n, lmbd, otimizacao)
            
            if test_data:
                acc = self.evaluate(test_data)
                print("    Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));
   
            else:
                print("    Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂, len_training_data, lmbd=0, otimizacao=None, momentum=0.8):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""
        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
          #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
          #soma os deltas do minibatch
          delta_nabla_b, delta_nabla_w = self.backprop(x, y)
          nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
          nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
        n = len_training_data
        mb = len(mini_batch)
        if otimizacao=="L1":
          #https://github.com/meetvora/mlp-classifier/blob/master/neuralnet.py
          self.weights = [w-𝜂*lmbd*np.sign(w)/n-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.biases = [b-(𝜂/mb)*nb for b, nb in zip(self.biases, nabla_b)]
        elif otimizacao=="L2":
          #https://visualstudiomagazine.com/articles/2017/09/01/neural-network-l2.aspx (An Alternative Approach)
          #self.weights = [(1-(𝜂/mb)*(lmbd/mb))*w-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.weights = [(1-𝜂*(lmbd/n))*w-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.biases = [b-(𝜂/mb)*nb for b, nb in zip(self.biases, nabla_b)]
        elif otimizacao=="MOMENTO":
          #https://github.com/Kanav123/MNIST-Neural-Network/blob/bb0fad846c6193f1272fba496a26217e589467d0/network2.py
          vel_w = [np.zeros(w.shape) for w in self.weights]
          vel_b = [np.zeros(b.shape) for b in self.biases]
          vel_w = [(momentum*v) - (𝜂/mb)*nw for v, nw in zip(vel_w, nabla_w)]
          vel_b = [(momentum*v) - (𝜂/mb)*nb for v, nb in zip(vel_b, nabla_b)]
          self.weights = [(v + (1- 𝜂*(lmbd/n))*w) for w ,v in zip(self.weights, vel_w)]
          self.biases = [(b + v) for b, v in zip(self.biases, vel_b)]
        else:
          self.weights = [w-(𝜂/mb)*nw for w, nw in zip(self.weights, nabla_w)]
          self.biases = [b-(𝜂/mb)*nb for b, nb in zip(self.biases, nabla_b)]


    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores net, camada por camada
        nets = [] 

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b   
            nets.append(net)
            activation = sigmoid(net) #z = valor de saída do neurônio
            activations.append(activation)
        
        # Backward pass 
        
        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥
        
        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante. 
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = sigmoid_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [ ]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)
test_data = list(test_data)

𝜂 = 0.3
arquitecture = [784, 40, 10] 
print("\n1. NET={}, ETA={}".format(arquitecture,𝜂))
print("  MLP")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data)

print("  otimizacao='L1'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.01, otimizacao="L1")

print("  otimizacao='L2'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="L2")

print("  otimizacao='MOMENTO'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="MOMENTO", momentum=0.8)



1. NET=[784, 30, 20, 10], ETA=0.5
  MLP
    Epoch 0 : 6048 / 10000 = 60.48%
    Epoch 1 : 7830 / 10000 = 78.3%
    Epoch 2 : 8281 / 10000 = 82.81%
    Epoch 3 : 8490 / 10000 = 84.9%
    Epoch 4 : 8676 / 10000 = 86.76%
    Epoch 5 : 8763 / 10000 = 87.63%
    Epoch 6 : 8840 / 10000 = 88.4%
    Epoch 7 : 8894 / 10000 = 88.94%
    Epoch 8 : 8933 / 10000 = 89.33%
    Epoch 9 : 8966 / 10000 = 89.66%
    Epoch 10 : 8997 / 10000 = 89.97%
    Epoch 11 : 9036 / 10000 = 90.36%
    Epoch 12 : 9057 / 10000 = 90.57%
    Epoch 13 : 9102 / 10000 = 91.02%
    Epoch 14 : 9103 / 10000 = 91.03%
    Epoch 15 : 9114 / 10000 = 91.14%
    Epoch 16 : 9149 / 10000 = 91.49%
    Epoch 17 : 9174 / 10000 = 91.74%
    Epoch 18 : 9176 / 10000 = 91.76%
    Epoch 19 : 9173 / 10000 = 91.73%
  otimizacao='L1'
    Epoch 0 : 6629 / 10000 = 66.29%
    Epoch 1 : 7724 / 10000 = 77.24%
    Epoch 2 : 8258 / 10000 = 82.58%
    Epoch 3 : 8519 / 10000 = 85.19%
    Epoch 4 : 8686 / 10000 = 86.86%
    Epoch 5 : 8771 / 10000 = 87.71

In [ ]:
𝜂 = 0.5
arquitecture = [784, 30, 20, 10] 
print("\n2. NET={}, ETA={}".format(arquitecture,𝜂))

print("  MLP")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data)

print("  otimizacao='L1'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.01, otimizacao="L1")

print("  otimizacao='L2'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="L2")

print("  otimizacao='MOMENTO'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="MOMENTO", momentum=0.8)



2. NET=[784, 20, 10], ETA=0.5
  MLP
    Epoch 0 : 6370 / 10000 = 63.7%
    Epoch 1 : 7342 / 10000 = 73.42%
    Epoch 2 : 7635 / 10000 = 76.35%
    Epoch 3 : 7845 / 10000 = 78.45%
    Epoch 4 : 8448 / 10000 = 84.48%
    Epoch 5 : 8624 / 10000 = 86.24%
    Epoch 6 : 8743 / 10000 = 87.43%
    Epoch 7 : 8795 / 10000 = 87.95%
    Epoch 8 : 8850 / 10000 = 88.5%
    Epoch 9 : 8880 / 10000 = 88.8%
    Epoch 10 : 8924 / 10000 = 89.24%
    Epoch 11 : 8948 / 10000 = 89.48%
    Epoch 12 : 8968 / 10000 = 89.68%
    Epoch 13 : 8991 / 10000 = 89.91%
    Epoch 14 : 9010 / 10000 = 90.1%
    Epoch 15 : 9043 / 10000 = 90.43%
    Epoch 16 : 9050 / 10000 = 90.5%
    Epoch 17 : 9059 / 10000 = 90.59%
    Epoch 18 : 9069 / 10000 = 90.69%
    Epoch 19 : 9073 / 10000 = 90.73%
  otimizacao='L1'
    Epoch 0 : 6682 / 10000 = 66.82%
    Epoch 1 : 7896 / 10000 = 78.96%
    Epoch 2 : 8292 / 10000 = 82.92%
    Epoch 3 : 8510 / 10000 = 85.1%
    Epoch 4 : 8645 / 10000 = 86.45%
    Epoch 5 : 8701 / 10000 = 87.01%
    E

In [ ]:
𝜂 = 0.3
arquitecture = [784, 30, 10]
print("\n3. NET={}, ETA={}".format(arquitecture,𝜂))

print("  MLP")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data)

print("  otimizacao='L1'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.01, otimizacao="L1")

print("  otimizacao='L2'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="L2")

print("  otimizacao='MOMENTO'")
mlp = Network(arquitecture)
mlp.SGD(training_data, 20, 32, 𝜂, test_data=test_data, lmbd=0.10, otimizacao="MOMENTO", momentum=0.8)


3. NET=[784, 30, 20, 10], ETA=0.3
  MLP
    Epoch 0 : 4594 / 10000 = 45.94%
    Epoch 1 : 6365 / 10000 = 63.65%
    Epoch 2 : 7270 / 10000 = 72.7%
    Epoch 3 : 7754 / 10000 = 77.54%
    Epoch 4 : 8060 / 10000 = 80.6%
    Epoch 5 : 8286 / 10000 = 82.86%
    Epoch 6 : 8409 / 10000 = 84.09%
    Epoch 7 : 8502 / 10000 = 85.02%
    Epoch 8 : 8600 / 10000 = 86.0%
    Epoch 9 : 8662 / 10000 = 86.62%
    Epoch 10 : 8728 / 10000 = 87.28%
    Epoch 11 : 8776 / 10000 = 87.76%
    Epoch 12 : 8814 / 10000 = 88.14%
    Epoch 13 : 8860 / 10000 = 88.6%
    Epoch 14 : 8901 / 10000 = 89.01%
    Epoch 15 : 8916 / 10000 = 89.16%
    Epoch 16 : 8956 / 10000 = 89.56%
    Epoch 17 : 8965 / 10000 = 89.65%
    Epoch 18 : 8990 / 10000 = 89.9%
    Epoch 19 : 8994 / 10000 = 89.94%
  otimizacao='L1'
    Epoch 0 : 4941 / 10000 = 49.41%
    Epoch 1 : 6418 / 10000 = 64.18%
    Epoch 2 : 7287 / 10000 = 72.87%
    Epoch 3 : 7830 / 10000 = 78.3%
    Epoch 4 : 8131 / 10000 = 81.31%
    Epoch 5 : 8334 / 10000 = 83.34%
 

##**Mini-Projeto**

A partir dos melhores resultados obtidos no projeto anterior, execute 3 configurações pra cada questão a seguir:
<pre>
1º 𝜂 = 0.5, net = [784, 30, 20, 10] = 91.73% 
2º 𝜂 = 0.5, net = [784, 20, 10] = 90.73%
3º 𝜂 = 0.3, net = [784, 30, 20, 10] = 89.94%
</pre>

1) Implementar as regularizações L1 e L2

2) Implementar o Momento

3) Comparar os experimentos e explicar o porquê de cada resultado. Qual foi a melhor regularização? Por que? O Momento melhorou os resultados? Por que? 
<blockquote>
<p>
O melhor resultado obtido foi: <strong>92.15%</strong>, na configuração 1 (𝜂 = 0.5, net = [784, 30, 20, 10]), utilizando o Momento. Mas é bom registrar que todos os resultados foram melhores que os originais.
</p>
<p>
Quanto a regularização, os resultados variaram com as configurações das redes, não sendo possível observar "a melhor" entre o L1 e o L2. No entanto, para a configuração 1 (melhor configuração), o L1 foi mais eficaz, apresentando um valor de 91.82%.
</p>
<p>
Um fato observado é que a adição do <strong>momento</strong> melhorou todos os resultados. Uma vez que esta otimização evita os mínimos locais, pode ser possível atingir o mínimo global (ou melhores mínimos).
</p>
</blockquote>

Complete a Tabela abaixo com os resultados 

\begin{array}{|c|c|c|c|}\hline\\ \\
  Configuracao & 1 & 2 & 3 \\ \hline 
L1 & 91.82 & 90.95 & 90.42  \\ \hline
L2  & 91.74 & 91.35 & 90.41 \\ \hline 
Momento  & 92.15 & 92.07 & 90.47 \\ \hline
\end{array}



\\
Data de Entrega: 23/12/2020
